In [16]:
!pip3 install gensim
!pip3 install spacy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Inicialmente se va a realizar un preprocesado de los datos, eliminando las palabras sin significado útil, los url y los signos de puntuación.

In [17]:
import pandas as pd
import numpy as np
import gensim as gensim
from gensim.models import Word2Vec
import spacy
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score


df = pd.read_csv('./data/train.csv')

In [18]:
# Preprocesamiento del texto
!python3 -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')

Defaulting to user installation because normal site-packages is not writeable
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.7.1/en_core_web_lg-3.7.1-py3-none-any.whl (587.7 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


Preprocesamiento corrigiendo ortografia y eliminando URLs, puntuacion, emojis y stopwords

In [ ]:
import re
import string
from spellchecker import SpellChecker
en_stopwords = nlp.Defaults.stop_words

#Function to correct spelling errors
def correct_spellings(text):
    spell = SpellChecker()
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(filter(None, corrected_text))
        
#Function to remove URLs
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

#Function to remove emojis from the text
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

#Function to remove punctuation
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

#Function to remove stopwords from the text
def remove_stop_words(text):
    cleanText = ''
    phrase = nlp(text)
    for token in phrase:
        if not token.is_stop:
        #and not token.is_punct and not token.like_url:
            cleanText += ' ' + token.text
    return cleanText

#df['text']=df['text'].apply(lambda x : correct_spellings(x))
df['text']=df['text'].apply(lambda x : remove_URL(x))
#df['text']=df['text'].apply(lambda x : remove_emoji(x))
df['text']=df['text'].apply(lambda x : remove_punct(x))
df['text']=df['text'].apply(remove_stop_words)

df['text_cleaned'] = df['text'].apply(lambda x: gensim.utils.simple_preprocess(x))

In [ ]:
df['text_cleaned']

Division del dataset en train y test

In [ ]:
# Separar datos de entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text_cleaned'], df["label"], test_size=0.33, random_state=42)

Entrenamiento del modelo de vectorización

In [ ]:
word2vec_model = Word2Vec(X_train, vector_size=100, window=5, min_count=2)

words = set(word2vec_model.wv.index_to_key )
X_train_vect = np.array([np.array([word2vec_model.wv[i] for i in ls if i in words])
                         for ls in X_train])
X_test_vect = np.array([np.array([word2vec_model.wv[i] for i in ls if i in words])
                         for ls in X_test])

In [ ]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

Entrenamiento del modelo de clasificación

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV

word2vec_pipe = Pipeline([('estimator', LogisticRegression())])

# fit pipeline
word2vec_pipe.fit(X_train_vect_avg, y_train.values.ravel())

Optimizacion de parametros

In [ ]:
param_grid = {
    'estimator__penalty': ['l1','l2'], 
    'estimator__C': [100, 10, 1.0, 0.1, 0.01],
    'estimator__solver': ['newton-cg', 'lbfgs', 'liblinear']
}

word2vec_grid_search = GridSearchCV(
    estimator = word2vec_pipe,
    param_grid = param_grid,
    scoring = 'f1',
    n_jobs = -1,
    refit = True,
    verbose = 1,
    return_train_score = True
)

# fit RandomizedSearchCV object
word2vec_grid_search.fit(X_train_vect_avg, y_train.values.ravel())

pd.options.display.max_colwidth = 50

word2vec_grid_search_results = pd.DataFrame(word2vec_grid_search.cv_results_).sort_values(by = 'mean_test_score', ascending = False)

Prediccion:

In [ ]:
predictions = word2vec_grid_search.predict(X_test_vect_avg)

Resultados:

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, predictions))

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
from sklearn import metrics
accuracy = metrics.accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)